In [1]:
import pandas as pd
import numpy as np
import utils
import torch
import preprocess_text
import joblib

In [3]:
# dir(utils)

In [4]:
utils.convert_z_to_csv('Data/test_1.z', 'Data/test_1.csv')
utils.convert_z_to_csv('Data/test_2.z', 'Data/test_2.csv')
utils.convert_z_to_csv('Data/test_3.z', 'Data/test_3.csv')

## Xử lý data categorical

In [177]:
def dummies(data):
    label_each_aspect = list(range(6))
#     data = data.rename(columns={"vui chơi": "a0", "lưu trú": "a1", "nhà hàng": "a2", 
#                                "ăn uống": "a3", "vận chuyển": "a4", "mua sắm": "a5"})
    data = data.rename(columns={"giai_tri": "a0", "luu_tru": "a1", "nha_hang": "a2", 
                               "an_uong": "a3", "di_chuyen": "a4", "mua_sam": "a5"})
    columns = data.columns
    dummys = []
    for cl in columns:
        if cl != 'Review':
            for label in label_each_aspect:
                data[f'{cl}_{label}'] = data.apply(lambda row: 0 if row[cl] != label else 1, axis=1)
                
    data = data.drop(["a0","a1","a2","a3","a4","a5" ], axis =1)
    return data

In [184]:
datafile = 'Data/processed_data_final.csv'
datafile = 'Data/processed_data_final_remake.csv'
df = pd.read_csv(datafile)

columns = df.columns.tolist()

df = df[columns[:7]]

In [185]:
df.head(5)

,Review,giai_tri,luu_tru,nha_hang,an_uong,di_chuyen,mua_sam
0,"Bánh rất nhiều tôm to, tôm giòn nằm chễm chệ t...",0,0,0,5,0,0
1,Đi Quy Nhơn cứ lo không có pizza ăn nhưng nhờ ...,0,0,0,5,0,0
2,"các món ăn ở đây không có gì đặc sắc, có lẽ nấ...",0,0,1,1,0,0
3,"Lần thứ 2 quay lại đây, vì giới thiệu quán cà ...",0,0,0,2,0,0
4,Rất khuyên bạn nên ghé thăm quán bar thể thao này,3,0,0,0,0,0


In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3658 entries, 0 to 3657
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     3658 non-null   object
 1   giai_tri   3658 non-null   int64 
 2   luu_tru    3658 non-null   int64 
 3   nha_hang   3658 non-null   int64 
 4   an_uong    3658 non-null   int64 
 5   di_chuyen  3658 non-null   int64 
 6   mua_sam    3658 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 200.2+ KB


In [187]:
print(df[df['Review'].isnull()].index.tolist())
print(df[df['giai_tri'].isnull()].index.tolist())
print(df[df['luu_tru'].isnull()].index.tolist())
print(df[df['nha_hang'].isnull()].index.tolist())
print(df[df['an_uong'].isnull()].index.tolist())
print(df[df['di_chuyen'].isnull()].index.tolist())
print(df[df['mua_sam'].isnull()].index.tolist())

[]
[]
[]
[]
[]
[]
[]


In [188]:
print(len(df['Review'].tolist()))
print(len(set(df['Review'].tolist())))

3658
3658


In [158]:
# x = df['Review'].tolist()
# z = dict()
# for i in x:
#     if i in z:
#         z[i] += 1
#         print(i)
#         break
#     else:
#         z[i] = 1 
# with open('Data/duplicate.txt', 'w') as f:
#     for i in z:
#         if z[i] > 1:
#             f.write(i+'\n')
# f.close()


In [183]:
new_df = dummies(df)
new_df.head()

,Review,a0_0,a0_1,a0_2,a0_3,a0_4,a0_5,a1_0,a1_1,a1_2,...,a4_2,a4_3,a4_4,a4_5,a5_0,a5_1,a5_2,a5_3,a5_4,a5_5
0,"Nơi đó thật kinh khủng, chúng tôi nhìn thấy nh...",1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,"Quán sạch sẽ, món ăn tươi ngon, giá lại rẻ",1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,"Phục vụ mãi mãi, thức ăn không ngon, và họ thậ...",1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,"Nhiều món đăc trưng Vú nàng nướng mỡ hành, bọ ...",1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Không phải là một cách tồi để đi giữa Sài Gòn ...,1,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [160]:
len(set(new_df.Review.tolist()))

3665

In [161]:
# store_file = 'Data/data_final.z'
store_file = 'Data/data_final_remake.z'
# df = pd.read_csv(data_file, delimiter=',', header=0, encoding="utf8")
data = new_df.values.tolist()
print(len(data))
# print(data[0][0])
# feature = [preprocess_text.preprocessing_sample(x[0]) for x in data]
feature = [x[0] for x in data]
label = [x[1:] for x in data]
data = [[fe,lb] for fe,lb in zip(feature,label)]
import joblib
joblib.dump(data, store_file)

3665


['Data/data_final_remake.z']

In [1]:
# len(features)

In [99]:
import joblib
def read_data(data_file):
    if '.csv' in data_file:
        df = pd.read_csv(data_file, delimiter=',', header=0, encoding="utf8")
        data = []
        for sample in df.values.tolist():
            data.append([sample[0], sample[1:]])
        return data
    else:
        return joblib.load(data_file)

In [100]:
data = read_data('Data_Add/train_1000.z')

In [101]:
len(data)

1000

In [102]:
labels = [i[1] for i in data]

In [103]:
import numpy as np
x = np.array(labels)

In [104]:
x.shape

(1000, 36)

In [105]:
x[1] - x[2]

array([ 0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  1, -1,  0,  0,  0,  0,
        1,  0, -1,  0,  0,  0,  1, -1,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        0,  0])

In [106]:
import torch
labels = torch.tensor(np.array(labels), dtype=torch.float)

In [107]:
train_data = read_data('Data/clean-train.z')
#     if args.aug_train_folder:
#     if args.add_aug_data:
#         for f in os.listdir(args.aug_train_folder):
#             train_data.extend(read_data(args.aug_train_folder+f))
train_data.extend(read_data('Data_Add/train_1000.z'))

In [108]:
len(train_data)

3965

In [109]:
labels = []
for sample in train_data:
    labels.append(sample[1])

In [110]:
labels = torch.tensor(np.array(labels), dtype=torch.float)

In [1]:
import joblib
data = joblib.load('Data/clean-train.z')

In [4]:
sample = data[0]
logit = sample[1]

In [6]:
aspect_logit = []
score_logit = []
for i in range(0, 36, 6):
    x = logit[i:i+6].index(max(logit[i:i+6]))
    if x > 0:
        aspect_logit.append(1)
    else:
        aspect_logit.append(0)
    score_logit.append(x)

In [8]:
score_logit

[0, 0, 0, 5, 0, 0]

## Oversampling

In [28]:
import pandas as pd
train_df = pd.read_csv('Data/train-problem2.csv')

In [29]:
import numpy as np
len_data = []
for i in range(6):
    len_data.append(len(train_df[train_df[f"a{i}"] >0]))

max_field = np.argmax(len_data)
max_count = max(len_data)

In [30]:
train_df[(train_df[f"a{max_field}"] ==0) & (train_df[f'a{0}'] >0)]

,Review,a0,a1,a2,a3,a4,a5
4,Rất khuyên bạn nên ghé thăm quán bar thể thao này,3,0,0,0,0,0
7,Tôi bắt xe 7 chỗ đi từ sân bay về nhà.Thái độ ...,1,0,0,0,1,0
8,Bãi Đá Trứng có những viên đá hình tròn hoặc b...,5,0,0,0,0,0
18,"Hiện tại đã tu sửa xong, tuy nhiên loại hình ...",4,0,0,0,0,0
26,"Mát mẻ,nhìn cũng dc,nhưng ko có gì đặc sắc cho...",4,0,0,0,0,0
...,...,...,...,...,...,...,...
2949,Nhận và giao hàng ổn nhưng thông báo trước 1 n...,4,0,0,0,4,0
2950,"Không khí trên quán bar thì khỏi chê rồi, nhạc...",4,0,0,0,0,0
2951,"Safari chỉ có tầm 14 loài thôi, mà khỉ là nhiề...",3,0,0,0,0,0
2956,Là địa điểm tốt cho các tín đồ sống ảo: bãi bi...,4,0,0,0,0,0


In [31]:
unique_field = []
for i in range(6):
    if i != max_field:
        unique_field.append(train_df[(train_df[f"a{max_field}"] ==0) & (train_df[f'a{i}'] >0)])
    if i == max_field:
        unique_field.append(train_df[train_df[f'a{i}']> 0])

In [32]:
len(unique_field)

6

In [33]:
for i in range(len(unique_field)):
    unique_field[i] = unique_field[i].reset_index(drop= True)
unique_field[0]

,Review,a0,a1,a2,a3,a4,a5
0,Rất khuyên bạn nên ghé thăm quán bar thể thao này,3,0,0,0,0,0
1,Tôi bắt xe 7 chỗ đi từ sân bay về nhà.Thái độ ...,1,0,0,0,1,0
2,Bãi Đá Trứng có những viên đá hình tròn hoặc b...,5,0,0,0,0,0
3,"Hiện tại đã tu sửa xong, tuy nhiên loại hình ...",4,0,0,0,0,0
4,"Mát mẻ,nhìn cũng dc,nhưng ko có gì đặc sắc cho...",4,0,0,0,0,0
...,...,...,...,...,...,...,...
364,Nhận và giao hàng ổn nhưng thông báo trước 1 n...,4,0,0,0,4,0
365,"Không khí trên quán bar thì khỏi chê rồi, nhạc...",4,0,0,0,0,0
366,"Safari chỉ có tầm 14 loài thôi, mà khỉ là nhiề...",3,0,0,0,0,0
367,Là địa điểm tốt cho các tín đồ sống ảo: bãi bi...,4,0,0,0,0,0


In [35]:
added = []
added.append(max_field)
len_data[max_field] = 0
while(sum(len_data) != 0):

    max_field_i = np.argmax(len_data)
    added.append(max_field_i)
    required_sample = max_count - np.max(len_data)
    chosen_idx = np.random.choice(len(unique_field[max_field_i]), replace = True, size =required_sample )
    add_df = unique_field[max_field_i].iloc[chosen_idx]
    break
    train_df = train_df.append(add_df,ignore_index=True)
#     train_df = pd.concat(train_df, add_df)
    break
    #### update param
    len_data[max_field_i] = 0
    for i in range(len(len_data)):
        if len_data[i] !=0:
            len_data[i] = len_data[i] + len(add_df[add_df[f'a{i}'] > 0])
            unique_field[i] = unique_field[i][unique_field[i][f'a{max_field_i}'] == 0]

In [36]:
add_df

,Review,a0,a1,a2,a3,a4,a5
0,"Phòng có giường lớn, êm đẹp, sạch sẽ,... Lễ tâ...",0,5,0,0,0,0
499,Đây là lần thứ 2 quay lại chỗ này. Ban đầu mìn...,0,5,0,0,0,0
176,Toilet bố trí không phù hợp lắm. Gương lớn chi...,0,3,0,0,0,0
546,Phòng gia đình nhưng bị tách thành 2 phòng ghé...,0,3,0,0,0,0
127,Thiếu nhiều vật dung cơ bản: khăn giấy trên ph...,0,1,0,0,0,0
...,...,...,...,...,...,...,...
567,"Chất lượng dịch vụ tại fleur de lys oke, nhân ...",0,5,0,0,0,0
535,Buổi tối có hơi ồn một tí! Nhưng vẫn ok.,0,4,0,0,0,0
435,"Ks gần biển, phòng mình có view hướng biển cả ...",0,4,0,0,0,0
276,địa chỉ trên booking cần cụ thể hơn. mình mất ...,0,3,0,0,0,0


In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3497 entries, 0 to 3496
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  3497 non-null   object
 1   a0      3497 non-null   int64 
 2   a1      3497 non-null   int64 
 3   a2      3497 non-null   int64 
 4   a3      3497 non-null   int64 
 5   a4      3497 non-null   int64 
 6   a5      3497 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 191.4+ KB


In [8]:
len_data = []
for i in range(6):
    len_data.append(len(train_df[train_df[f'a{i}']>0]))

In [9]:
len_data

[1177, 1177, 1177, 1177, 1177, 1177]

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6492 entries, 0 to 6491
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  6492 non-null   object
 1   a0      6492 non-null   int64 
 2   a1      6492 non-null   int64 
 3   a2      6492 non-null   int64 
 4   a3      6492 non-null   int64 
 5   a4      6492 non-null   int64 
 6   a5      6492 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 355.2+ KB


In [11]:
train_df.head()

,Review,a0,a1,a2,a3,a4,a5
0,"Bánh rất nhiều tôm to, tôm giòn nằm chễm chệ t...",0,0,0,5,0,0
1,Đi Quy Nhơn cứ lo không có pizza ăn nhưng nhờ ...,0,0,0,5,0,0
2,"các món ăn ở đây không có gì đặc sắc, có lẽ nấ...",0,0,1,1,0,0
3,"Lần thứ 2 quay lại đây, vì giới thiệu quán cà ...",0,0,0,2,0,0
4,Rất khuyên bạn nên ghé thăm quán bar thể thao này,3,0,0,0,0,0


## So sanh file goc va file sua

In [162]:
def convert_logit_to_score(logit):
    res = []
    for i in range(0, 36, 6):
        x = logit[i:i+6]
        res.append(x.index(max(x)))
    return res

In [163]:
org_data = joblib.load('Data/data_final.z')
remake_data = joblib.load('Data/data_final_remake.z')
org_df = pd.read_csv('Data/processed_data_final.csv')
remake_df = pd.read_csv('Data/processed_data_final_remake.csv')
columns = remake_df.columns.tolist()[:7]
remake_df = remake_df[columns]

In [164]:
print(len(org_data))
print(len(remake_data))

3658
3665


In [117]:
# res = []
# for i in remake_data:
#     x = [i[0]]
#     x += convert_logit_to_score(i[1])
#     res.append(x)
# zzz = pd.DataFrame(res, columns=columns)
# zzz.to_csv('Data/processed_data_final_remake.csv')

In [165]:
review_in_org_data = [i[0] for i in org_data]
review_in_remake_data = [i[0] for i in remake_data]

In [166]:
print(len(set(review_in_org_data)))
print(len(set(review_in_remake_data)))

3658
3665


In [216]:
com_reviews = list(set(review_in_org_data).intersection(review_in_remake_data))
print(len(com_reviews))

3511


In [217]:
common_samples = []
for r in com_reviews:
    idx_1, idx_2 = review_in_org_data.index(r), review_in_remake_data.index(r)
    if utils.compare_arrays(org_data[idx_1][1], remake_data[idx_2][1]):
        common_samples.append([r, org_data[idx_1][1]])

In [218]:
len(common_samples)

2508

In [196]:
# common_samples[0]

In [194]:
# only_origin_samples, only_remake_samples = [], []
# for r in com_reviews:
#     idx_1, idx_2 = review_in_org_data.index(r), review_in_remake_data.index(r)
#     if r not in 
#     if utils.compare_arrays(org_data[idx_1][1], remake_data[idx_2][1]):
#         common_samples.append([r, org_data[idx_1][1]])

## Build data so sánh chất lượng data gốc và data sửa

In [ ]:
# # store_file = 'Data/data_final.z'
# store_file = 'Data/data_train_1.z'
# feature = [x[0] for x in data]
# label = [x[1:] for x in data]
# data = [[fe,lb] for fe,lb in zip(feature,label)]
# import joblib
# joblib.dump(data, store_file)

In [227]:
test_data = common_samples[1000:-1000]
test_review = [i[0] for i in test_data]

In [228]:
org_train_data, remake_train_data = [], []
for sample in org_data:
    if sample[0] not in test_review:
        org_train_data.append(sample)
        
for sample in remake_data:
    if sample[0] not in test_review:
        remake_train_data.append(sample)

In [229]:
len(org_train_data)

3150

In [230]:
# org_train_data[0]

In [231]:
len(remake_train_data)

3157

In [232]:
store_file = 'Data/org_train_3.z'
data = org_train_data
feature = [x[0] for x in data]
label = [x[1] for x in data]
data = [[fe,lb] for fe,lb in zip(feature,label)]
import joblib
joblib.dump(data, store_file)

['Data/org_train_3.z']

In [233]:
store_file = 'Data/remake_train_3.z'
data = remake_train_data
feature = [x[0] for x in data]
label = [x[1] for x in data]
data = [[fe,lb] for fe,lb in zip(feature,label)]
import joblib
joblib.dump(data, store_file)

['Data/remake_train_3.z']

In [234]:
store_file = 'Data/test_3.z'
data = test_data
feature = [x[0] for x in data]
label = [x[1] for x in data]
data = [[fe,lb] for fe,lb in zip(feature,label)]
import joblib
joblib.dump(data, store_file)

['Data/test_3.z']

In [215]:
# data[0]